# Campo de temperatura a 2m y líneas de corriente 

(Última actualización 10 abr 2023)

Ejemplo para graficar un campo de temperatura a 2m y líneas de corriente. <br />
*Example to plot a 2-m temperatura and streamlines.*[texto del enlace](https://)

In [ ]:
# En caso de utilizar Google colab, descomentar las siguientes líneas

#!pip install numpy==1.24  
#!pip install --no-binary shapely shapely --force
#!pip install h5netcdf
#!pip install s3fs
#!pip install cartopy
#!pip install metpy

In [2]:
# Importamos las librerías necesarias  (en caso de ser necesario, instalar utilizando el comando pip install)
# We import the necessary libraries    (if needed, install by using the command pip install)
import xarray as xr
import h5netcdf
import datetime
import s3fs
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
import metpy.calc as mpcalc

Definimos la fecha y hora de inicialización del pronóstico: <br />
*We define the forecast initialization date:* 

In [4]:
init_year = 2022
init_month = 4
init_day = 1
init_hour = 0
INIT_DATE = datetime.datetime(init_year, init_month, init_day, init_hour)

Definimos el plazo de pronóstico a graficar: <br />
*We define the forecast lead time to plot:*

In [10]:
lead_time = 12

Leemos el archivo: <br />
*We read the file:*

In [ ]:
filename = 'WRFDETAR_01H_{:%Y%m%d_%H}_{:03d}.nc'.format(INIT_DATE,lead_time) 
print(filename)
ds = xr.open_dataset(filename, decode_coords = 'all', engine = 'h5netcdf')

Calculamos las componentes zonal y meridional del viento a 10m: <br />
*We calculate the zonal and meridional components of 10-m wind:*




In [22]:
u, v = mpcalc.wind_components(ds['magViento10'], ds['dirViento10'])

u = u.metpy.dequantify()
v = v.metpy.dequantify()

Generamos la figura: <br />
*We generate the plot:*

In [ ]:
# Seleccionamos la proyección de los datos
# We chose a map projection
projection = ccrs.LambertConformal(central_longitude = ds['Lambert_Conformal'].attrs['longitude_of_central_meridian'], 
                                 central_latitude = ds['Lambert_Conformal'].attrs['latitude_of_projection_origin'], 
                                 standard_parallels = ds['Lambert_Conformal'].attrs['standard_parallel'], cutoff = -5)

fig, ax = plt.subplots(1, 1, figsize = (7, 10), constrained_layout = True, subplot_kw = {'projection': projection})
ax.set_extent(extents=[-500*4000, 500*4000, -600*4000, 600*4000], crs = projection)
cs = ax.pcolormesh(ds['lon'], ds['lat'], ds['T2'][0, :, :], vmin = -15, vmax = 30, cmap = 'inferno', transform = ccrs.PlateCarree())
strm = ax.streamplot(ds['lon'].values, ds['lat'].values, u[0, :, :].values, v[0, :, :].values, density = [1.5, 1.5], color = 'white', linewidth = 0.75, transform = ccrs.PlateCarree())
ax.add_feature(cf.COASTLINE)
ax.add_feature(cf.BORDERS)
ax.set_title(f'2-m Temperature and Streamlines')
gl = ax.gridlines(crs = ccrs.PlateCarree(), draw_labels = True, x_inline = False,
                  linewidth = 2, color = 'gray', alpha = 0.5, linestyle = '--')
gl.top_labels = False
gl.right_labels = False
cbar = plt.colorbar(cs, shrink = 0.6)
cbar.set_label('[°C]', rotation=90)
plt.show()